In [74]:
import keras.backend as K
from keras.layers import Input, Dense, Concatenate, Flatten, Dropout, Conv3D, MaxPooling3D, ZeroPadding3D
from keras.models import Model
from keras.regularizers import l1, l2
from keras.optimizers import Adam
from keras.constraints import max_norm
from keras.layers.noise import GaussianNoise
from keras.callbacks import EarlyStopping
from keras.utils import np_utils
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score

import cnn_methods as cfunc
import config
import csv
import helper_fxns as hf
import importlib
import matplotlib.pyplot as plt
import numpy as np
import operator
import os
import pandas as pd
import random
import re
%matplotlib inline

In [75]:
importlib.reload(config)
importlib.reload(cfunc)
C = config.Config()

In [76]:
dims = C.dims
nb_classes = len(C.classes_to_include)

voi_img = Input(shape=(dims[0], dims[1], dims[2], C.nb_channels))
x = voi_img
#x = GaussianNoise(1)(x)
#x = ZeroPadding3D(padding=(3,3,2))(voi_img)
x = Conv3D(filters=128, kernel_size=(3,3,2), activation='relu')(x)
x = Dropout(0.5)(x)
x = Conv3D(filters=128, kernel_size=(3,3,2), activation='relu')(x)
x = MaxPooling3D((2, 2, 2))(x)
x = Dropout(0.5)(x)
#x = Conv3D(filters=64, kernel_size=(3,3,2), strides=(2, 2, 2), activation='relu', kernel_constraint=max_norm(4.))(x)
#x = Dropout(0.5)(x)
x = Conv3D(filters=64, kernel_size=(3,3,2), activation='relu')(x)
x = MaxPooling3D((2, 2, 1))(x)
x = Dropout(0.5)(x)
x = Flatten()(x)

img_traits = Input(shape=(2,)) #bounding volume and aspect ratio of lesion

intermed = Concatenate(axis=1)([x, img_traits])
x = Dense(64, activation='relu')(intermed)#, kernel_initializer='normal', kernel_regularizer=l1(.01), kernel_constraint=max_norm(3.))(x)
x = Dropout(0.5)(x)
pred_class = Dense(nb_classes, activation='softmax')(x)#Dense(nb_classes, activation='softmax')(x)

In [ ]:
dims = C.dims
nb_classes = len(C.classes_to_include)

voi_img = Input(shape=(dims[0], dims[1], dims[2], C.nb_channels))
x = voi_img
#x = GaussianNoise(1)(x)
#x = ZeroPadding3D(padding=(3,3,2))(voi_img)
x = Conv3D(filters=128, kernel_size=(3,3,2), activation='relu')(x)
x = Dropout(0.5)(x)
x = Conv3D(filters=128, kernel_size=(3,3,2), activation='relu')(x)
x = MaxPooling3D((2, 2, 2))(x)
x = Dropout(0.5)(x)
#x = Conv3D(filters=64, kernel_size=(3,3,2), strides=(2, 2, 2), activation='relu', kernel_constraint=max_norm(4.))(x)
#x = Dropout(0.5)(x)
x = Conv3D(filters=64, kernel_size=(3,3,2), activation='relu')(x)
x = MaxPooling3D((2, 2, 1))(x)
x = Dropout(0.5)(x)
x = Flatten()(x)

img_traits = Input(shape=(2,)) #bounding volume and aspect ratio of lesion

intermed = Concatenate(axis=1)([x, img_traits])
x = Dense(64, activation='relu')(intermed)#, kernel_initializer='normal', kernel_regularizer=l1(.01), kernel_constraint=max_norm(3.))(x)
x = Dropout(0.5)(x)
pred_class = Dense(nb_classes, activation='softmax')(x)

optim = Adam(lr=0.01)#5, decay=0.001)
early_stopping = EarlyStopping(monitor='acc', min_delta=0.01, patience=4)

model_pretrain = Model([voi_img, img_traits], pred_class)
model_pretrain.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_pretrain.summary()

In [77]:
optim = Adam(lr=0.01)#5, decay=0.001)
early_stopping = EarlyStopping(monitor='acc', min_delta=0.01, patience=4)

model = Model([voi_img, img_traits], pred_class)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_9 (InputLayer)             (None, 36, 36, 12, 3) 0                                            
____________________________________________________________________________________________________
conv3d_11 (Conv3D)               (None, 34, 34, 11, 12 7040        input_9[0][0]                    
____________________________________________________________________________________________________
dropout_14 (Dropout)             (None, 34, 34, 11, 12 0           conv3d_11[0][0]                  
____________________________________________________________________________________________________
conv3d_12 (Conv3D)               (None, 32, 32, 10, 12 295040      dropout_14[0][0]                 
___________________________________________________________________________________________

model_pretrain = Model([voi_img, img_traits], pred_class)
for l in range(1,5):
    if type(model_pretrain.layers[l]) == Conv3D:
        model_pretrain.layers[l].set_weights(model.layers[l].get_weights())

In [78]:
voi_df = pd.read_csv(C.art_voi_path)
intensity_df = pd.read_csv(C.int_df_path)
#intensity_df.loc[intensity_df["art_int"] == 0, "art_int"] = np.mean(intensity_df[intensity_df["art_int"] > 0]["art_int"])
#intensity_df.loc[intensity_df["ven_int"] == 0, "ven_int"] = np.mean(intensity_df[intensity_df["ven_int"] > 0]["ven_int"])
#intensity_df.loc[intensity_df["eq_int"] == 0, "eq_int"] = np.mean(intensity_df[intensity_df["eq_int"] > 0]["eq_int"])

orig_data_dict, num_samples = cfunc.collect_unaug_data(C.classes_to_include, C, voi_df, intensity_df)
print(num_samples)

{'fnh': 115, 'cyst': 77, 'hcc': 54, 'hemangioma': 60, 'colorectal': 70, 'cholangio': 61}


In [79]:
train_ids = {} #filenames of training set originals
test_ids = {} #filenames of test set
X_test = []
X2_test = []
Y_test = []
Z_test = []
X_train_orig = []
X2_train_orig = []
Y_train_orig = []
Z_train_orig = []
cls_mapping = []

train_samples = {}

for cls_num, cls in enumerate(orig_data_dict):
    cls_mapping.append(cls)
    
    train_samples[cls] = round(num_samples[cls]*C.train_frac)
    
    order = np.random.permutation(list(range(num_samples[cls])))
    train_ids[cls] = list(orig_data_dict[cls][2][order[:train_samples[cls]]])
    test_ids[cls] = list(orig_data_dict[cls][2][order[train_samples[cls]:]])
    
    X_test = X_test + list(orig_data_dict[cls][0][order[train_samples[cls]:]])
    X2_test = X2_test + list(orig_data_dict[cls][1][order[train_samples[cls]:]])
    Y_test = Y_test + [[0] * cls_num + [1] + [0] * (nb_classes - cls_num - 1)] * \
                        (num_samples[cls] - train_samples[cls])
    Z_test = Z_test + test_ids[cls]
    
    X_train_orig = X_train_orig + list(orig_data_dict[cls][0][order[:train_samples[cls]]])
    X2_train_orig = X2_train_orig + list(orig_data_dict[cls][1][order[:train_samples[cls]]])
    Y_train_orig = Y_train_orig + [[0] * cls_num + [1] + [0] * (nb_classes - cls_num - 1)] * \
                        (train_samples[cls])
    Z_train_orig = Z_train_orig + train_ids[cls]
    
    print("%s has %d samples for training (%d after augmentation) and %d for testing" %
          (cls, train_samples[cls], train_samples[cls] * C.aug_factor, num_samples[cls] - train_samples[cls]))
    
#Y_test = np_utils.to_categorical(Y_test, nb_classes)
#Y_train_orig = np_utils.to_categorical(Y_train_orig, nb_classes)

fnh has 92 samples for training (4600 after augmentation) and 23 for testing
cyst has 62 samples for training (3100 after augmentation) and 15 for testing
hcc has 43 samples for training (2150 after augmentation) and 11 for testing
hemangioma has 48 samples for training (2400 after augmentation) and 12 for testing
colorectal has 56 samples for training (2800 after augmentation) and 14 for testing
cholangio has 49 samples for training (2450 after augmentation) and 12 for testing


def scale_to_one(X, scale_by_channel=True):
    def scale_img(x):
        return 2 * (x - np.amin(x)) / (np.amax(x) - np.amin(x)) - 1
    return np.array(X)

    X = np.array(X)
    
    if scale_by_channel:
        X = [np.stack([scale_img(x[:,:,:,0]), scale_img(x[:,:,:,1]), scale_img(x[:,:,:,2])], axis=3) for x in X]
    else:
        X = [scale_img(x) for x in X]
    
    return np.array(X)

In [80]:
#scale from -1 to 1
#X_train = [scale_to_one(X_train), tbd(X2_train)]
#X_val = [scale_to_one(X_val), tbd(X2_val)]
X_test = [np.array(X_test), np.array(X2_test)]
X_train_orig = [np.array(X_train_orig), np.array(X2_train_orig)]

#Y_train = np.array(Y_train)
#Y_val = np.array(Y_val)
Y_test = np.array(Y_test)
Y_train_orig = np.array(Y_train_orig)

#Z_train = np.array(Z_train)
Z_test = np.array(Z_test)
Z_train_orig = np.array(Z_train_orig)

In [81]:
avg_X2 = {}
for cls in C.classes_to_include:
    avg_X2[cls] = np.mean(orig_data_dict[cls][1], axis=0)

In [95]:
def train_generator_func(n=12, n_art=0):
    """n is the number of samples from each class, n_art is the number of artificial samples"""
    classes_to_include = C.classes_to_include
    
    num_classes = len(classes_to_include)
    while True:
        x1 = np.empty(((n+n_art)*num_classes, dims[0], dims[1], dims[2], C.nb_channels))
        x2 = np.empty(((n+n_art)*num_classes, 2))
        y = np.zeros(((n+n_art)*num_classes, num_classes))

        train_cnt = 0
        for cls in classes_to_include:
            img_fns = os.listdir(C.aug_dir+cls)
            while n>0:
                img_fn = random.choice(img_fns)
                if img_fn[:img_fn.rfind('_')] + ".npy" in train_ids[cls]:
                    x1[train_cnt] = np.load(C.aug_dir+cls+"\\"+img_fn)
                    x1[train_cnt] = cfunc.rescale_int(x1[train_cnt],
                                          intensity_df[intensity_df["AccNum"] == img_fn[:img_fn.find('_')]])

                    row = voi_df[(voi_df["Filename"] == img_fn[:img_fn.find('_')] + ".npy") &
                                 (voi_df["lesion_num"] == int(img_fn[img_fn.find('_')+1:img_fn.rfind('_')]))]
                    x2[train_cnt] = [(float(row["real_dx"]) * float(row["real_dy"]) * float(row["real_dz"])) ** (1/3) / 50,
                                        max(float(row["real_dx"]), float(row["real_dy"])) / float(row["real_dz"])]
                    
                    y[train_cnt][cls_mapping.index(cls)] = 1
                    
                    train_cnt += 1
                    if train_cnt % n == 0:
                        break
            
                        
            if n_art>0:
                img_fns = os.listdir(C.artif_dir+cls)
                for _ in range(n_art):
                    img_fn = random.choice(img_fns)
                    x1[train_cnt] = np.load(C.artif_dir + cls + "\\" + img_fn)
                    x2[train_cnt] = avg_X2[cls]
                    y[train_cnt][cls_mapping.index(cls)] = 1

                    train_cnt += 1
        
        yield [np.array(x1), np.array(x2)], np.array(y) #cfunc.separate_phases([np.array(x1), np.array(x2)]), np.array(y)

In [96]:
train_generator = train_generator_func()
model_pretrain.fit_generator(train_generator, steps_per_epoch=120, epochs=50)#, callbacks=[early_stopping])

Epoch 1/50
120/120 [==============================] - 62s - loss: 4.6349 - acc: 0.2622    
Epoch 2/50
120/120 [==============================] - 53s - loss: 2.2526 - acc: 0.4429    
Epoch 3/50
120/120 [==============================] - 50s - loss: 2.0645 - acc: 0.5128    
Epoch 4/50
120/120 [==============================] - 48s - loss: 1.9554 - acc: 0.5236    
Epoch 5/50
120/120 [==============================] - 48s - loss: 1.8532 - acc: 0.5488    
Epoch 6/50
120/120 [==============================] - 50s - loss: 1.9207 - acc: 0.5529    
Epoch 7/50
120/120 [==============================] - 48s - loss: 2.0199 - acc: 0.5549    
Epoch 8/50
120/120 [==============================] - 48s - loss: 1.8439 - acc: 0.5667    
Epoch 9/50
120/120 [==============================] - 48s - loss: 1.7345 - acc: 0.5954    
Epoch 10/50
120/120 [==============================] - 49s - loss: 1.6613 - acc: 0.5922    
Epoch 11/50
120/120 [==============================] - 49s - loss: 1.6947 - acc: 0.6019  

In [83]:
train_generator = train_generator_func()
model.fit_generator(train_generator, steps_per_epoch=120, epochs=50)#, callbacks=[early_stopping])

Epoch 1/50
120/120 [==============================] - 82s - loss: 0.5249 - acc: 0.7742    
Epoch 2/50
120/120 [==============================] - 82s - loss: 0.2833 - acc: 0.8921    
Epoch 3/50
120/120 [==============================] - 82s - loss: 0.2265 - acc: 0.9127    
Epoch 4/50
120/120 [==============================] - 84s - loss: 0.1883 - acc: 0.9306    
Epoch 5/50
120/120 [==============================] - 84s - loss: 0.1987 - acc: 0.9282    
Epoch 6/50
120/120 [==============================] - 85s - loss: 0.1667 - acc: 0.9409    
Epoch 7/50
120/120 [==============================] - 85s - loss: 0.1461 - acc: 0.9472    
Epoch 8/50
120/120 [==============================] - 85s - loss: 0.1502 - acc: 0.9488    
Epoch 9/50
120/120 [==============================] - 85s - loss: 0.1323 - acc: 0.9528    
Epoch 10/50
120/120 [==============================] - 85s - loss: 0.0985 - acc: 0.9655    
Epoch 11/50
120/120 [==============================] - 85s - loss: 0.0919 - acc: 0.9670  

KeyboardInterrupt: 

hist = model.fit(X_train2, Y_train2, batch_size=32, epochs=200, validation_data=(X_test, Y_test), callbacks=[early_stopping])

In [66]:
with open(C.small_voi_path, 'r') as csv_file:
    reader = csv.reader(csv_file)
    small_vois = dict(reader)
    
for key in small_vois:
    small_vois[key] = [int(x) for x in small_vois[key][1:-1].split(', ')]

In [67]:
voi_df_art = pd.read_csv(C.art_voi_path)
voi_df_ven = pd.read_csv(C.ven_voi_path)
voi_df_eq = pd.read_csv(C.eq_voi_path)

In [68]:
cls_mapping

['fnh', 'cyst', 'hcc', 'hemangioma', 'colorectal', 'cholangio']

In [50]:
X_test = cfunc.separate_phases(X_test)
X_train_orig = cfunc.separate_phases(X_train_orig)

In [98]:
#plot_with_bbox(fn_list[2], cls_mapping[wrong_guesses[2]])
Y_pred = model_pretrain.predict(X_test)
y_true = np.array([max(enumerate(x), key=operator.itemgetter(1))[0] for x in Y_test])
y_pred = np.array([max(enumerate(x), key=operator.itemgetter(1))[0] for x in Y_pred])

cm = confusion_matrix(y_true, y_pred)
f1 = f1_score(y_true, y_pred, average="weighted")

#cfunc.save_output(Z_test, y_pred, y_true, voi_df_art, small_vois, cls_mapping, C)

In [73]:
Y_pred = model.predict(X_train_orig)
y_true = np.array([max(enumerate(x), key=operator.itemgetter(1))[0] for x in Y_train_orig])
y_pred = np.array([max(enumerate(x), key=operator.itemgetter(1))[0] for x in Y_pred])

cm = confusion_matrix(y_true, y_pred)
f1 = f1_score(y_true, y_pred, average="weighted")

cfunc.save_output(Z_train_orig, y_pred, y_true, voi_df_art, small_vois, cls_mapping, C)

In [63]:
cls_mapping

['colorectal', 'hemangioma', 'hcc', 'cyst', 'cholangio']

In [33]:
cm

array([[18,  0,  0,  3,  1],
       [ 1, 19,  0,  2,  2],
       [ 1,  3, 26,  0,  2],
       [ 1,  3,  0, 14,  6],
       [ 9,  6,  0,  2, 11]], dtype=int64)

In [71]:
y_true_simp, y_pred_simp, _ = cfunc.condense_cm(y_true, y_pred, cls_mapping)

In [72]:
accuracy_score(y_true_simp, y_pred_simp)

0.71264367816091956

In [99]:
accuracy_score(y_true, y_pred)

0.40229885057471265

In [ ]:
4 cls: 94/94%, 86/88%, 86%/90%
5 cls: 80%/89%, 72%/86%
6 cls: 60%/71%

In [365]:
fn_list = Z_test[~np.equal(y_pred, y_true)]
wrong_guesses = np.array(y_pred)[~np.equal(y_pred, y_true)]

In [409]:
Z_test[~np.equal(y_pred, y_true)]

array(['E100815762_0.npy', '12239513_0.npy', '12042703_2.npy'],
      dtype='<U16')

In [21]:
voi_df_art[voi_df_art["Filename"] == "12673726.npy"]

,Filename,x1,x2,y1,y2,z1,z2,cls,flipz,dx,dy,dz,id
119,12673726.npy,105,134,76,101,40,53,hemangioma,True,29,25,13,126737261057640
120,12673726.npy,127,139,103,117,44,51,hemangioma,True,12,14,7,1267372612710344


In [22]:
voi_df_ven[voi_df_ven["id"] == "1267372612710344"]

,id,x1,x2,y1,y2,z1,z2,dx,dy,dz
74,1267372612710344,127,139,103,117,44,52,12,14,8


activ = model2.predict(X_train)
#activ = model2.predict(np.expand_dims(X_train[10],axis=0))

In [316]:
import operator
#print("Ground truth:", [cls_mapping[max(enumerate(x), key=operator.itemgetter(1))[0]] for x in Y_val[::30]])
Y_ = model.predict(X_test)
print("Predictions:", [cls_mapping[max(enumerate(x), key=operator.itemgetter(1))[0]] + " (%.5f%% confidence)" % (max(x)*100) for x in Y_])

Predictions: ['hcc (95.75861% confidence)', 'hcc (99.98515% confidence)', 'hcc (56.77554% confidence)', 'hcc (97.68384% confidence)', 'hcc (96.73136% confidence)', 'hcc (99.99979% confidence)', 'hcc (99.99988% confidence)', 'hcc (99.71866% confidence)', 'cyst (63.61439% confidence)', 'cyst (51.41591% confidence)', 'hcc (99.65365% confidence)', 'hcc (52.72004% confidence)', 'hcc (57.44457% confidence)', 'cyst (99.82330% confidence)', 'cyst (83.69393% confidence)']


In [16]:
C.classes_to_include = ['cyst', 'hcc', 'hemangioma', 'cholangio', 'colorectal']#, 'fnh']
dims = C.dims
nb_classes = len(C.classes_to_include)

art_img = Input(shape=(dims[0], dims[1], dims[2], 1))
art_x = art_img
art_x = Conv3D(filters=64, kernel_size=(3,3,2), activation='relu')(art_x)
art_x = Dropout(0.5)(art_x)
art_x = MaxPooling3D((2, 2, 2))(art_x)

                                
ven_img = Input(shape=(dims[0], dims[1], dims[2], 1))
ven_x = ven_img
ven_x = Conv3D(filters=64, kernel_size=(3,3,2), activation='relu')(ven_x)
ven_x = Dropout(0.5)(ven_x)
ven_x = MaxPooling3D((2, 2, 2))(ven_x)

eq_img = Input(shape=(dims[0], dims[1], dims[2], 1))
eq_x = eq_img
eq_x = Conv3D(filters=64, kernel_size=(3,3,2), activation='relu')(eq_x)
eq_x = Dropout(0.5)(eq_x)
eq_x = MaxPooling3D((2, 2, 2))(eq_x)

intermed = Concatenate(axis=4)([art_x, ven_x, eq_x])
x = Conv3D(filters=100, kernel_size=(3,3,2), activation='relu')(intermed)
x = MaxPooling3D((2, 2, 1))(x)
x = Dropout(0.5)(x)
x = Flatten()(x)

img_traits = Input(shape=(2,)) #bounding volume and aspect ratio of lesion

intermed = Concatenate(axis=1)([x, img_traits])
x = Dense(100, activation='relu')(intermed)#, kernel_initializer='normal', kernel_regularizer=l1(.01), kernel_constraint=max_norm(3.))(x)
x = Dropout(0.5)(x)
pred_class = Dense(nb_classes, activation='softmax')(x)#Dense(nb_classes, activation='softmax')(x)

In [17]:
optim = Adam(lr=0.01)#5, decay=0.001)
early_stopping = EarlyStopping(monitor='acc', min_delta=0.01, patience=4)

model = Model([art_img, ven_img, eq_img, img_traits], pred_class)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary(line_length=120)

________________________________________________________________________________________________________________________
Layer (type)                           Output Shape               Param #       Connected to                            
input_3 (InputLayer)                   (None, 36, 36, 12, 1)      0                                                     
________________________________________________________________________________________________________________________
input_4 (InputLayer)                   (None, 36, 36, 12, 1)      0                                                     
________________________________________________________________________________________________________________________
input_5 (InputLayer)                   (None, 36, 36, 12, 1)      0                                                     
________________________________________________________________________________________________________________________
conv3d_4 (Conv3D)               

In [86]:
a=time.time()
Y_ = model.predict(X_val)
print(time.time()-a)

0.20054101943969727


In [136]:
import matplotlib.pyplot as plt
%matplotlib inline